<a href="https://colab.research.google.com/github/dnkhiem/course-resources-ml-with-experts-budgets/blob/master/khiem_test_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
datalink = 'https://github.com/ttruongatl/ml-lstm/blob/master/data/29297294_02-08-2019.csv'

!wget --no-check-certificate \
    https://raw.githubusercontent.com/ttruongatl/ml-lstm/master/data/mil-week-labeled-train.csv \
    -O /tmp/mil-week-labeled-train.csv
!pip install tensorflow==2.0.0

--2019-12-11 03:22:56--  https://raw.githubusercontent.com/ttruongatl/ml-lstm/master/data/mil-week-labeled-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95752027 (91M) [text/plain]
Saving to: ‘/tmp/mil-week-labeled-train.csv’

/tmp/mil-week-label 100%[===================>]  91.32M   156MB/s    in 0.6s    

2019-12-11 03:23:01 (156 MB/s) - ‘/tmp/mil-week-labeled-train.csv’ saved [95752027/95752027]



In [0]:
import json
import tensorflow as tf
import csv
import random
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

In [0]:
num_sentences = 0
datetimes = []
datas = []
labels = []
colume = [1,2,3,4,5,6,7,8]
header = ['timestamp', 'CylinderBorePressure', 'CylinderRodPressure', 'RotationCWPressure', 'RotationCCWPressure', 'MotionX', 'MotionY', 'MotionZ', 'StringPotV', 'event', 'start']
label_dict = {}
count = 0
with open("/tmp/mil-week-labeled-train.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
      data = []
      for col in colume:
        data.append(float(row[col]))
      datas.append(data)
      datetimes.append(row[0])
      labels.append(row[9])
      if row[9] not in label_dict:
        label_dict[row[9]] = count
        count += 1
datas = np.array(datas)
labels = np.array(list(label_dict[x] for x in labels))

In [4]:
print(label_dict)
print(datas.shape)

{'idle': 0, 'cut': 1, 'sort': 2}
(900025, 8)


In [0]:
def windowed_dataset(data_in, windows = 20):
  data_extend = np.zeros((data_in.shape[0] + windows, data_in.shape[1]))
  data_extend[windows:,:] = data_in[:,:]
  data_extend = data_extend.T
  data_out = np.zeros((data_in.shape[1], data_in.shape[0], windows))
  print(data_extend.shape)
  print(data_out.shape)
  for i in range(data_in.shape[0]):  
    data_out[:,i,:] = data_extend[:,i:i+windows]
  return data_out

In [6]:
from keras.utils import to_categorical

window_size = 40
n_samples = 10000

datas_new = windowed_dataset(datas, windows = window_size)
data_samples = datas_new[:, :n_samples, :] #Get 20 k
labels = labels[:n_samples]

split_time = int(n_samples *0.8)

#max_X = np.max(datas_new)
X_train = data_samples[:,:split_time,:]
y_train = to_categorical(labels[:split_time])

X_valid = data_samples[:,split_time:,:]
y_valid = to_categorical(labels[split_time:])


X_train = np.transpose(X_train, (1, 2, 0)) / 6000
X_valid = np.transpose(X_valid, (1, 2, 0)) / 6000

print(X_train.shape)
print(X_valid.shape)


Using TensorFlow backend.


(8, 900065)
(8, 900025, 40)
(8000, 40, 8)
(2000, 40, 8)


In [7]:
print(X_train.shape)
print(X_valid.shape)
np.max(X_train)



(8000, 40, 8)
(2000, 40, 8)


0.867325

In [0]:
def separate_block(input_tensor, n_LSTMs = 16, n_Dense = 16):
  x = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_tensor)
  x = tf.keras.layers.Conv1D(n_LSTMs, 5, activation = 'relu') (x) # test with Conv1D
# x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.LSTM(n_LSTMs, return_sequences=True)(x)
  x = tf.keras.layers.LSTM(n_LSTMs)(x)
  x = tf.keras.layers.Dense(n_Dense, activation="relu")(x)
  return x

In [0]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.99 and logs.get('val_acc')>0.95):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

In [0]:
tf.keras.backend.clear_session()
#tf.random.set_seed(51)
#np.random.seed(51)
#window_size = 64
#batch_size = 256
window_size = 40
batch_size = 32
epochs = 200
callback = myCallback()


input_data = tf.keras.layers.Input(shape = [None, 8])
print(input_data)

x0 = separate_block(input_data[:,:, 0], 32, 32)
x1 = separate_block(input_data[:,:, 1], 32, 32)
x2 = separate_block(input_data[:,:, 2], 32, 32)
x3 = separate_block(input_data[:,:, 3], 32, 32)
x4 = separate_block(input_data[:,:, 4], 16, 16)
x5 = separate_block(input_data[:,:, 5], 16, 16)
x6 = separate_block(input_data[:,:, 6], 16, 16)
x7 = separate_block(input_data[:,:, 7], 16, 16)
x = tf.keras.layers.concatenate([x0, x1, x2, x3, x4, x5, x6, x7])
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(3, activation = 'softmax')(x)

model = tf.keras.models.Model(input_data, x)

#model = tf.keras.models.Sequential([
#  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
#                      input_shape=[None]),
#  tf.keras.layers.Conv1D(32, 5, activation = 'relu'),  # test with Conv1D
#  tf.keras.layers.Dropout(0.2),
#  tf.keras.layers.LSTM(32, return_sequences=True),
#  tf.keras.layers.LSTM(32),
#  tf.keras.layers.Dense(16, activation="relu"),
#  tf.keras.layers.Dense(3, activation = 'softmax')
#])
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=["acc"])
history = model.fit(X_train, y_train, epochs=epochs, batch_size= batch_size,
                    validation_data=(X_valid, y_valid), verbose = 1, callbacks = [callback])

Tensor("input_1:0", shape=(None, None, 8), dtype=float32)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 8)]    0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, None)]       0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, None)]       0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_2 (Te [(None, None)]       0           input_1[0][0]                    
____________________________________

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()